<a href="https://colab.research.google.com/github/RiyadAhmedCS/CSNEA2022/blob/main/QuoteGenerationAI_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
%cd "/content/drive/MyDrive/nea_files/CaptionGeneration/"
%ls
!pip install git+git://github.com/huggingface/transformers/

/content/drive/MyDrive/nea_files/CaptionGeneration
 QuoteDatasetPrep.ipynb            quotes_dataset.csv   train_m
'QuoteGenerationAI System.ipynb'   test_m
  Cloning git://github.com/huggingface/transformers/ to /tmp/pip-req-build-ygvgvqaq
  Running command git clone -q git://github.com/huggingface/transformers/ /tmp/pip-req-build-ygvgvqaq
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [20]:
from transformers import AutoTokenizer
from transformers import AutoModelWithLMHead
from transformers import TextDataset
from transformers import DataCollatorForLanguageModeling

tokenizer = AutoTokenizer.from_pretrained('gpt2') 
model = AutoModelWithLMHead.from_pretrained('gpt2')

loading configuration file https://huggingface.co/gpt2/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/fc674cd6907b4c9e933cb42d67662436b89fa9540a1f40d7c919d0109289ad01.7d2e0efa5ca20cef4fb199382111e9d3ad96fd77b849e1d4bed13a66e1336f51
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {


In [ ]:
def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset("train_m","test_m",tokenizer)


/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,
Creating features from dataset file at 
Token indices sequence length is longer than the specified maximum sequence length for this model (17819 > 1024). Running this sequence through the model will result in indexing errors
Saving features into cached file cached_lm_GPT2TokenizerFast_128_train_m [took 0.010 s]
Creating features from dataset file at 
Saving features into cached file cached_lm_GPT2TokenizerFast_128_test_m [took 0.009 s]


In [22]:
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    output_dir="./gpt2-motivational_v6", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=32, # number of training epochs
    per_gpu_train_batch_size=4, # batch size for training
    per_gpu_eval_batch_size=8,  # batch size for evaluation
    logging_steps = 500, # Number of update steps between two evaluations.
    save_steps=500, # after # steps model is saved
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [23]:
%env PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:1024


env: PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:1024


In [24]:
import torch
torch.cuda.empty_cache()

trainer.train() 
trainer.save_model("./gpt2-motivational_v6") 
tokenizer.save_pretrained("./gpt2-motivational_v6")

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
***** Running training *****
  Num examples = 139
  Num Epochs = 32
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 1120
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_eval_batch_size` argument which will be removed in a future version. Using `--per_device_eval_batch_size` is preferred.


Step,Training Loss
500,2.924600
1000,0.865000


Saving model checkpoint to ./gpt2-motivational_v6/checkpoint-500
Configuration saved in ./gpt2-motivational_v6/checkpoint-500/config.json
Model weights saved in ./gpt2-motivational_v6/checkpoint-500/pytorch_model.bin
Saving model checkpoint to ./gpt2-motivational_v6/checkpoint-1000
Configuration saved in ./gpt2-motivational_v6/checkpoint-1000/config.json
Model weights saved in ./gpt2-motivational_v6/checkpoint-1000/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to ./gpt2-motivational_v6
Configuration saved in ./gpt2-motivational_v6/config.json
Model weights saved in ./gpt2-motivational_v6/pytorch_model.bin
tokenizer config file saved in ./gpt2-motivational_v6/tokenizer_config.json
Special tokens file saved in ./gpt2-motivational_v6/special_tokens_map.json


('./gpt2-motivational_v6/tokenizer_config.json',
 './gpt2-motivational_v6/special_tokens_map.json',
 './gpt2-motivational_v6/vocab.json',
 './gpt2-motivational_v6/merges.txt',
 './gpt2-motivational_v6/added_tokens.json',
 './gpt2-motivational_v6/tokenizer.json')

In [25]:
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("./gpt2-motivational_v6")
model = AutoModelWithLMHead.from_pretrained("./gpt2-motivational_v6")

gpt2_finetune = pipeline('text-generation', model=model, tokenizer=tokenizer)
gpt2_finetune("")

# gen_kwargs has different options like max_length, 
# beam_search options, top-p, top-k, etc

for i in range(1,10):
  gpt2_finetune("")


Didn't find file ./gpt2-motivational_v6/added_tokens.json. We won't load it.
loading file ./gpt2-motivational_v6/vocab.json
loading file ./gpt2-motivational_v6/merges.txt
loading file ./gpt2-motivational_v6/tokenizer.json
loading file None
loading file ./gpt2-motivational_v6/special_tokens_map.json
loading file ./gpt2-motivational_v6/tokenizer_config.json
/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:788: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
loading configuration file ./gpt2-motivational_v6/config.json
Model config GPT2Config {
  "_name_or_path": "./gpt2-motivational_v6",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "em

In [26]:
listofquotes = []

for i in range(0,20):
  quote = gpt2_finetune("",clean_up_tokenization_spaces=True)
  gentext = quote[0]["generated_text"]
  if (len(gentext) != 0) and ("-" not in gentext):
    listofquotes.append(gentext)
  listofquotes = [i.rstrip() for i in listofquotes]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

to become a better you, you have to lift a person up.
sad story telling." ~ sivaprakash sidhu

the oldest of the oldest people in society.
This time is different. spend more time thinking and less time doing.
The beauty of life lies in the ways in which it confers comfort and happiness.
enemies of ours are enemies of ours too.
The difference between how you respond to something and what you do in it, is attitude.
the difference between failure and survival is that coping skill you used in between the struggles.
in the face of defeat, you rise from the depths of the grave and rise again.
there is no failure, there is only success.

if you find yourself in this situation, you are dangerous.
to inspire others. whether you know it or not, inspiration makes a difference.
do what is right, not what is easy nor what is popular.

resources are hired to give results, not reasons.


In [46]:
for i in listofquotes:
  skip = False
  for j in i:
    if (j.isalpha() == False) and (j != " ") and (j != ".") and (j != ",") and (j != "\n"):
      skip = True
      break
  if skip == False:
    print(i.replace("\n", ""))

to become a better you, you have to lift a person up.
This time is different. spend more time thinking and less time doing.
The beauty of life lies in the ways in which it confers comfort and happiness.
enemies of ours are enemies of ours too.
The difference between how you respond to something and what you do in it, is attitude.
the difference between failure and survival is that coping skill you used in between the struggles.
in the face of defeat, you rise from the depths of the grave and rise again.
there is no failure, there is only success.
if you find yourself in this situation, you are dangerous.
to inspire others. whether you know it or not, inspiration makes a difference.
do what is right, not what is easy nor what is popular.
resources are hired to give results, not reasons.
